In [0]:
%sh 
pip install geopandas

Requirement already satisfied: geopandas in /databricks/python3/lib/python3.8/site-packages (0.11.0)
Requirement already satisfied: pyproj>=2.6.1.post1 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (3.3.1)
Requirement already satisfied: shapely<2,>=1.7 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.2)
Requirement already satisfied: pandas>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.2.4)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from geopandas) (20.9)
Requirement already satisfied: fiona>=1.8 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.21)
Requirement already satisfied: six>=1.7 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.15.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from fiona>=1.8->geopandas) (52.0.0)
Requirement already satisfied: click>=4.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (8.1.3)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2020.12.5)
Requirement already satisfied: munch in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2.5.0)
Requirement already satisfied: cligj>=0.5 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (0.7.2)
Requirement already satisfied: click-plugins>=1.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.1.1)
Requirement already satisfied: attrs>=17 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (20.3.0)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (1.20.1)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (2.8.1)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging->geopandas) (2.4.7)
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import os
import geopandas as gpd
from pyspark.sql.functions import col, expr, when
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [0]:
# read the shapefile
polygons = gpd.read_file("/dbfs/data/cuspatial_data/polygons/polygons.shp")
polygons_df = spark.createDataFrame(
    polygons
)
polygons_df.createOrReplaceTempView("polygons")
polygons_df.count()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:340: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Did not pass numpy.dtype object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Out[9]: 4

In [0]:
point_parquet_df = spark.read.format("parquet").\
    load("dbfs:/data/cuspatial_data/points")

point_parquet_df.createOrReplaceTempView("pointtable")
point_df = spark.sql("select ST_Point(x, y) as mypoint from pointtable")
point_df.show(5)

+--------------------+
 mypoint|
+--------------------+
POINT (0.48171647...|
POINT (1.25917257...|
POINT (0.18952591...|
POINT (3.02836214...|
POINT (1.38906644...|
+--------------------+
only showing top 5 rows

In [0]:
result = spark.sql("SELECT polygons.*, pointtable.* FROM polygons , pointtable WHERE ST_Contains(polygons.geometry, ST_Point(pointtable.x, pointtable.y))")
result.show()

+--------------------+------------------+------------------+
 geometry| x| y|
+--------------------+------------------+------------------+
POLYGON ((2.08811...|2.1807636574967466|3.2296461832828114|
POLYGON ((2.08811...|2.2006520196663066|3.7672478678985257|
POLYGON ((2.08811...|2.5104987015171574|3.0668114607133137|
POLYGON ((2.08811...|2.3007438625108882|3.6045900851589048|
POLYGON ((2.08811...| 2.566986568683904|3.6607732238530897|
POLYGON ((2.08811...| 2.493975723955388|3.3999020934055837|
POLYGON ((2.08811...|2.8222482218882474|3.8159308233351266|
POLYGON ((2.08811...| 2.241538022180476|3.8812819070357545|
POLYGON ((2.08811...|1.8703303641352362| 4.209727933188015|
POLYGON ((2.48845...|2.4264509160270813| 5.188939408363776|
POLYGON ((5.03982...| 6.291790729917634| 2.983311357415729|
POLYGON ((5.03982...| 6.101327777646798|2.5239201807166616|
POLYGON ((5.03982...| 6.109985464455084|2.2235950639628523|
POLYGON ((5.03982...|6.4274219368674315|2.9754616970668213|
POLYGON ((5.03982...|6.0821276168848994|2.5470532680258002|
POLYGON ((5.03982...| 6.325158445513714|2.8765450351723674|
POLYGON ((5.03982...| 6.444584786789386| 2.174562817047202|
POLYGON ((5.03982...| 6.6793884701899|2.5605928243991434|
POLYGON ((5.03982...| 7.079453687660189| 3.063690547962938|
+--------------------+------------------+------------------+